In [23]:
# Import quandl to retrieve data
# Pandas and matplotlib standard imports for data analysis
import quandl
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Use Plotly for better graphs
import plotly.offline as plotly
import plotly.graph_objs as go
import plotly.graph_objects as go

# Offline Plotting
plotly.init_notebook_mode(connected=True)

# Put your api key here
quandl.ApiConfig.api_key = 'g3sVyGQgGrx_J1RV9E43'
mapbox_access_token = 'pk.eyJ1IjoiY2hyaXN0aW5hdGhpcCIsImEiOiJjazB4ZGN2N3YwNTJuM21sYXd3eHZ1dmJiIn0.ogLmQDqwezijg16HL_9RTQ'

In [46]:
# Specify Zillow Indicator we want to graph
indicator = 'MLPAH'

# Specify the county and state we want to generate the map for
metro = 'Nashville'
state = 'TN'

# Start date and end date should be the same
start_date = '2010-06-30'
end_date = '2018-06-30'

In [72]:
# Get data for each zip code in the city and add that to a single dataframe
def get_city_data(cities, indicator):
    df_array = []
    for city, code in cities.items():
        quandl_db_code = f'ZILLOW/Z{code}_{indicator}'
        data = quandl.get(quandl_db_code, start_date=start_date, end_date=end_date)
        data.rename(index=str, columns={'Value': city}, inplace=True)
        data.index = pd.to_datetime(data.index, format='%Y/%m/%d')
        df_array.append(data)

    ret = pd.concat(df_array, axis=1)
    return ret

In [73]:
# Here we are going to figure out what zip codes are in the specified county
# We will also use the logitude / latitude data for plotting

ZIP_CODES = pd.read_csv('nash_zipcode.csv') 
ZIP_CODES.head()

,zip_code,latitude,longitude,city,state
0,37013,36.03765,-86.63697,Nashville,TN
1,37027,36.00159,-86.78068,Nashville,TN
2,37072,36.35495,-86.76214,Nashville,TN
3,37076,36.14312,-86.58965,Nashville,TN
4,37115,36.25536,-86.69827,Nashville,TN


In [74]:
# We will call our function to fetch the data from quandl
# There will be errors for data we cannot retrieve

real_estate_data = get_city_data(ZIP_CODES['zip_code'], indicator)
print(real_estate_data)
real_estate_data.to_csv(r'nashville.csv')


                  0         1         2         3         4         5   \
Date                                                                     
2010-06-30  138900.0  482400.0       NaN  159900.0  109900.0       NaN   
2010-07-31  137750.0  474950.0       NaN  154900.0  109900.0       NaN   
2010-08-31  134900.0  469900.0       NaN  152400.0  109900.0       NaN   
2010-09-30  135900.0  474990.0       NaN  153400.0  109500.0       NaN   
2010-10-31  135000.0  475000.0       NaN  157900.0  109000.0       NaN   
2010-11-30  135200.0  479000.0       NaN  155000.0  104950.0       NaN   
2010-12-31  134894.0  484950.0       NaN  154900.0   99900.0       NaN   
2011-01-31  130750.0  489900.0       NaN  153000.0  108500.0       NaN   
2011-02-28  131995.0  462445.0       NaN  149900.0   99900.0       NaN   
2011-03-31  129999.0  479000.0       NaN  149900.0   99900.0       NaN   
2011-04-30  129900.0  479500.0       NaN  149900.0   99900.0       NaN   
2011-05-31  129900.0  489900.0       N

In [65]:
# Here we are just doing some transformation and massaging of the data we get back
# If you want more information please run the notebook on your own

transformed_real_estate_data = real_estate_data.transpose()

column_name = f'{indicator} {end_date}'

In [51]:
transformed_real_estate_data[column_name] = transformed_real_estate_data[end_date].astype('float')

In [52]:
concat_data = pd.concat([ZIP_CODES, transformed_real_estate_data[column_name]], axis=1, sort=True)
#concat_data.to_csv(r'nashville.csv')
#print(concat_data)

    zip_code  latitude  longitude       city state  MLPAH 2018-06-30
0      37013  36.03765  -86.63697  Nashville    TN          230000.0
1      37027  36.00159  -86.78068  Nashville    TN          732450.0
2      37072  36.35495  -86.76214  Nashville    TN          269900.0
3      37076  36.14312  -86.58965  Nashville    TN          289341.0
4      37115  36.25536  -86.69827  Nashville    TN          205000.0
5      37138  36.24973  -86.61499  Nashville    TN          274900.0
6      37201  36.16596  -86.77321  Nashville    TN          379950.0
7      37203  36.15044  -86.79133  Nashville    TN          529900.0
8      37204  36.10797  -86.77619  Nashville    TN          699900.0
9      37205  36.11201  -86.96834  Nashville    TN          699900.0
10     37206  36.18013  -86.72929  Nashville    TN          409984.0
11     37207  36.23329  -86.78217  Nashville    TN          299999.0
12     37208  36.17574  -86.80616  Nashville    TN          350000.0
13     37209  36.15094  -86.90523 

In [53]:
# Now it's just a matter of plotting using the Plotly Mapbox graph
df = concat_data

plot_data = [
    go.Scattermapbox(
        lon = df['longitude'],
        lat = df['latitude'],
        mode='markers',
        marker=dict(
            reversescale = True,
            autocolorscale = False,
            size=17,
            opacity=0.85,
            colorscale = "rdylbu",
            cmin = df[column_name].min(),
            color = df[column_name],
            cmax = df[column_name].max(),
            colorbar=dict(
                title="Price"
            )
        ),
        #text = df['text'],
        #hoverinfo= 'text'
    )]

layout = go.Layout(
    title= f'{indicator} in {metro}, {state} {end_date}',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=df.iloc[0]['latitude'],
            lon=df.iloc[0]['longitude']
        ),
        pitch=0,
        zoom=8,
        style='light'
    ),
)

In [54]:
# Finally calling our plot function after declaring the layout

fig = dict( data=plot_data, layout=layout )
plotly.iplot( fig, filename=f'{indicator} {metro} {state} {end_date}')